In [1]:
import energyflow
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
X, Y = energyflow.datasets.qg_jets.load(num_data=100000, generator='pythia', pad=True, with_bc=False, cache_dir='~/.energyflow')

In [3]:
# preprocess by centering jets and normalizing pts
for x in X:
    mask = x[:,0] > 0
    yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)
    x[mask,1:3] -= yphi_avg
    x[mask,0] /= x[:,0].sum()

In [4]:
# preprocess PIDs so they are O(1) or less
X[:,:,3] = X[:,:,3] / 2000

In [5]:
nparticles = X.shape[1]

## Regular NN

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.c1 = nn.Conv1d(4, 16, 1)
        self.p1 = nn.AvgPool1d(nparticles)
        self.fc1 = nn.Linear(16, 16)
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.c1(x))
        x = self.p1(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
#         x = F.sigmoid(self.fc2(x))
#         x = F.logsigmoid(self.fc2(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)


Net(
  (c1): Conv1d(4, 16, kernel_size=(1,), stride=(1,))
  (p1): AvgPool1d(kernel_size=(139,), stride=(139,), padding=(0,))
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)


In [7]:
pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(pytorch_total_params)

369


In [60]:
X_torch = torch.tensor(X, dtype=torch.float).transpose(1, 2)

In [61]:
Y_torch = torch.tensor(Y, dtype=torch.float).unsqueeze(1)

In [62]:
class QGDataset(Dataset):
    """Quark/Gluon dataset from energyflow"""

    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


In [11]:
dataset = QGDataset(X_torch, Y_torch)
dataloader = DataLoader(dataset, batch_size=1,
                        shuffle=False)

In [19]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(net.parameters())

In [13]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # geat the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

/Users/smsharma/.conda/envs/Lensing-PowerSpectra/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,  2000] loss: 0.678
[1,  4000] loss: 0.582
[1,  6000] loss: 0.556
[1,  8000] loss: 0.541
[1, 10000] loss: 0.542
[1, 12000] loss: 0.530
[1, 14000] loss: 0.532
[1, 16000] loss: 0.523
[1, 18000] loss: 0.511
[1, 20000] loss: 0.513
[2,  2000] loss: 0.505
[2,  4000] loss: 0.500
[2,  6000] loss: 0.499
[2,  8000] loss: 0.487
[2, 10000] loss: 0.492
[2, 12000] loss: 0.487
[2, 14000] loss: 0.496
[2, 16000] loss: 0.487
[2, 18000] loss: 0.484
[2, 20000] loss: 0.490
Finished Training


In [14]:
# dataset_test = QGDataset(X_torch[:1000], Y_torch[:1000])
# dataloader_test = DataLoader(dataset_test, batch_size=5,
#                         shuffle=False)

In [15]:
def model_accuracy(net, data_x, data_y):
    # data_x and data_y are numpy matrices
    X = torch.Tensor(data_x)
    Y = torch.ByteTensor(data_y)   # a Tensor of 0s and 1s
    output = net(X)            # a Tensor of floats
    pred_y = output >= 0.5       # a Tensor of 0s and 1s
    num_correct = torch.sum(Y==pred_y)  # a Tensor
    acc = (num_correct.item() * 100.0 / len(data_y))  # scalar
    print('Accuracy: %d %%' % (acc))

In [16]:
model_accuracy(net, X_torch, Y_torch.byte())

Accuracy: 77 %


In [19]:
running_sum = 0
for p in net.parameters():
    if p.requires_grad:
        running_sum += p.numel()
        print(p.numel())
print(running_sum)

64
16
256
16
16
1
369


In [39]:
from torchsummary import summary
summary(net, input_size=(4, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1                [-1, 16, 1]              80
         AvgPool1d-2                [-1, 16, 1]               0
            Linear-3                   [-1, 16]             272
            Linear-4                    [-1, 1]              17
Total params: 369
Trainable params: 369
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


## Convert to BNN

In [155]:
import pyro
from pyro.distributions import Bernoulli, Normal, Delta
from pyro.infer import SVI, Trace_ELBO, TracePredictive
from pyro.optim import Adam
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal, AutoLowRankMultivariateNormal

In [139]:
net = Net()

In [140]:
dataset = QGDataset(X_torch, Y_torch)
dataloader = DataLoader(dataset, batch_size=32,
                        shuffle=False)

In [141]:
def model(x_data, y_data):
    
    c1w_prior = Normal(loc=torch.zeros_like(net.c1.weight), scale=torch.ones_like(net.c1.weight))
    c1b_prior = Normal(loc=torch.zeros_like(net.c1.bias), scale=torch.ones_like(net.c1.bias))

    fc1w_prior = Normal(loc=torch.zeros_like(net.fc1.weight), scale=torch.ones_like(net.fc1.weight))
    fc1b_prior = Normal(loc=torch.zeros_like(net.fc1.bias), scale=torch.ones_like(net.fc1.bias))

    fc2w_prior = Normal(loc=torch.zeros_like(net.fc2.weight), scale=torch.ones_like(net.fc2.weight))
    fc2b_prior = Normal(loc=torch.zeros_like(net.fc2.bias), scale=torch.ones_like(net.fc2.bias))
    
    priors = {'c1.weight': c1w_prior, 'c1.bias': c1b_prior,  'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior,  'fc2.weight': fc2w_prior, 'fc2.bias': fc2b_prior}
    
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", net, priors)
    lifted_model = lifted_module()
    
    model_out = lifted_model(x_data)
    
    pyro.sample("obs", Bernoulli(logits=model_out), obs=y_data)

In [142]:
guide = AutoLowRankMultivariateNormal(model)

In [143]:
torch_optim = Adam({"lr": 0.01})
svi = SVI(model, guide, torch_optim, loss=Trace_ELBO(num_particles=1))

In [144]:
# Test to make sure everything working
inputs, labels = next(iter(dataloader))
svi.step(inputs, labels)

303.76393699645996

In [146]:
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # geat the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        loss = svi.step(inputs, labels)

        # print statistics
        running_loss += loss
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200 / 32.) )
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 1.352
[1,   400] loss: 1.333
[1,   600] loss: 1.317
[1,   800] loss: 1.332
[1,  1000] loss: 1.279
[1,  1200] loss: 1.328
[1,  1400] loss: 1.326
[1,  1600] loss: 1.362
[1,  1800] loss: 1.310
[1,  2000] loss: 1.317
[1,  2200] loss: 1.353
[1,  2400] loss: 1.352
[1,  2600] loss: 1.293
[1,  2800] loss: 1.289
[1,  3000] loss: 1.359
[2,   200] loss: 1.341
[2,   400] loss: 1.380
[2,   600] loss: 1.336
[2,   800] loss: 1.297
[2,  1000] loss: 1.291
[2,  1200] loss: 1.321
[2,  1400] loss: 1.330
[2,  1600] loss: 1.348
[2,  1800] loss: 1.318
[2,  2000] loss: 1.343
[2,  2200] loss: 1.318
[2,  2400] loss: 1.358
[2,  2600] loss: 1.300
[2,  2800] loss: 1.299
[2,  3000] loss: 1.325
[3,   200] loss: 1.319
[3,   400] loss: 1.297
[3,   600] loss: 1.330
[3,   800] loss: 1.328
[3,  1000] loss: 1.365
[3,  1200] loss: 1.361
[3,  1400] loss: 1.312
[3,  1600] loss: 1.330
[3,  1800] loss: 1.335
[3,  2000] loss: 1.306
[3,  2200] loss: 1.340
[3,  2400] loss: 1.327
[3,  2600] loss: 1.357
[3,  2800] 

In [148]:
get_marginal = lambda traces, sites:EmpiricalMarginal(traces, sites)._get_samples_and_weights()[0].detach().cpu().numpy()

def summary(traces, sites):
    marginal = get_marginal(traces, sites)
    site_stats = {}
    for i in range(marginal.shape[1]):
        site_name = sites[i]
        marginal_site = pd.DataFrame(marginal[:, i]).transpose()
        describe = partial(pd.Series.describe, percentiles=[.05, 0.25, 0.5, 0.75, 0.95])
        site_stats[site_name] = marginal_site.apply(describe, axis=1) \
            [["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
    return site_stats

def wrapped_model(x_data, y_data):
    pyro.sample("prediction", Delta(model(x_data, y_data)))

posterior = svi.run(X_torch, Y_torch)


In [156]:
# posterior predictive distribution we can get samples from
trace_pred = TracePredictive(wrapped_model,
                             posterior,
                             num_samples=10)
post_pred = trace_pred.run(X_torch, None)
post_summary = summary(post_pred, sites= ['prediction', 'obs'])
mu = post_summary["prediction"]
y = post_summary["obs"]
predictions = pd.DataFrame({
    "cont_africa": X_torch[:, 0],
    "rugged": X_torch[:, 1],
    "mu_mean": mu["mean"],
    "mu_perc_5": mu["5%"],
    "mu_perc_95": mu["95%"],
    "y_mean": y["mean"],
    "y_perc_5": y["5%"],
    "y_perc_95": y["95%"],
    "true_gdp": Y_torch,
})


AttributeError: 'NoneType' object has no attribute 'dim'